In [14]:
import pandas as pd
import sys


In [16]:
sys.path.append('../../../../')  # Adiciona o diretório raiz ao path para importar conn.py
from conn import get_connection
engine = get_connection()


##### PDI

In [17]:
query_pdi = """
    SELECT * FROM byd.db_compras_byd """
df_pdi = pd.read_sql_query(query_pdi, engine)

C:\Users\giovanni.5683\AppData\Local\Temp\ipykernel_22500\982354752.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pdi = pd.read_sql_query(query_pdi, engine)


In [19]:
df_pdi

,origem,nome_empresa,chv_empresa,bandeira_bi,ordem_bandeira,sistema,revenda,idnfexterno,numero_nota_fiscal,serie_nota_fiscal,...,percentil_nps,percentil_visits,percentil_snapon,percentil_mystery,percentil_chatbot,percentil_download,dta_recebimento_pdi,dta_recebimento_atacado,dta_recebimento_performance,detalhes
0,APOLLO,25.1 BYD CURITIBA,A25_1,BYD,D,A,25,A251108535V010,10853,5,...,None,None,None,None,None,None,None,None,None,None
1,APOLLO,25.1 BYD CURITIBA,A25_1,BYD,D,A,25,A2511136745I010,113674,5,...,None,None,None,None,None,None,None,None,None,None
2,APOLLO,25.1 BYD CURITIBA,A25_1,BYD,D,A,25,A251105695I010,10569,5,...,None,None,None,None,None,None,None,None,None,None
3,APOLLO,25.1 BYD CURITIBA,A25_1,BYD,D,A,25,A251109205I010,10920,5,...,None,None,None,None,None,None,None,None,None,None
4,APOLLO,25.1 BYD CURITIBA,A25_1,BYD,D,A,25,A251109215I010,10921,5,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4197,APOLLO,25.9 BYD AUTOPLUS UMUARAMA,A25_9,BYD,D,A,25,A25991805V010,9180,5,...,None,None,None,None,None,None,None,None,None,None
4198,APOLLO,25.9 BYD AUTOPLUS UMUARAMA,A25_9,BYD,D,A,25,A25991815V010,9181,5,...,None,None,None,None,None,None,None,None,None,None
4199,APOLLO,25.9 BYD AUTOPLUS UMUARAMA,A25_9,BYD,D,A,25,A25991825V010,9182,5,...,None,None,None,None,None,None,None,None,None,None
4200,APOLLO,25.9 BYD AUTOPLUS UMUARAMA,A25_9,BYD,D,A,25,A25991835V010,9183,5,...,None,None,None,None,None,None,None,None,None,None


In [20]:
# Carregar o arquivo CSV
df_csv = pd.read_csv('../../../../Datasets/pdi.csv')

# Verificar quais chassis do CSV estão no df_pdi
chassis_csv = df_csv['chassi'].dropna().unique()
chassis_pdi = df_pdi['chassi'].dropna().unique()

# Encontrar os chassis que estão em ambos
chassis_comuns = set(chassis_csv) & set(chassis_pdi)

print(f"Total de chassis no CSV: {len(chassis_csv)}")
print(f"Total de chassis no df_pdi: {len(chassis_pdi)}")
print(f"Chassis que estão em ambos: {len(chassis_comuns)}")
print(f"\nPercentual de chassis do CSV presentes no df_pdi: {len(chassis_comuns) / len(chassis_csv) * 100:.2f}%")

Total de chassis no CSV: 469
Total de chassis no df_pdi: 4201
Chassis que estão em ambos: 469

Percentual de chassis do CSV presentes no df_pdi: 100.00%


In [23]:
# Encontrar chassi em comum
chassis_comuns = set(df_csv['chassi'].dropna()) & set(df_pdi['chassi'].dropna())
df_csv_comuns = df_csv[df_csv['chassi'].isin(chassis_comuns)].copy()

# Ajuste de tipos conforme necessário
df_csv_comuns['val_pdi'] = df_csv_comuns['val_pdi'].str.replace(',', '.').astype(float)
df_csv_comuns['dta_recebimento_pdi'] = pd.to_datetime(df_csv_comuns['dta_recebimento_pdi'], format='%d/%m/%Y')

# Defina as colunas que existem tanto no CSV quanto no banco
colunas_banco = set(df_pdi.columns)
colunas_csv = set(df_csv_comuns.columns)
colunas_comuns = list(colunas_banco & colunas_csv)
colunas_comuns.remove('chassi')  # 'chassi' será usado no WHERE

# Atualizar cada registro usando psycopg2
cursor = engine.cursor()
try:
    for _, row in df_csv_comuns.iterrows():
        set_clause = ', '.join([f"{col} = %s" for col in colunas_comuns])
        update_query = f"UPDATE byd.db_compras_byd SET {set_clause} WHERE chassi = %s"
        values = [row[col] for col in colunas_comuns] + [row['chassi']]
        cursor.execute(update_query, values)
    
    engine.commit()
    print(f"Atualização concluída! {len(df_csv_comuns)} registros atualizados.")
except Exception as e:
    engine.rollback()
    print(f"Erro durante atualização: {e}")
    raise
finally:
    cursor.close()

Atualização concluída! 469 registros atualizados.


##### ATACADO

In [24]:
query_atacado = """
    SELECT * FROM byd.db_compras_byd """
df_atacado = pd.read_sql_query(query_atacado, engine)

C:\Users\giovanni.5683\AppData\Local\Temp\ipykernel_22500\167893553.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_atacado = pd.read_sql_query(query_atacado, engine)


In [25]:
# Carregar o arquivo CSV
df_csv_atacado = pd.read_csv('../../../../Datasets/atacado.csv')

# Verificar quais chassis do CSV estão no df_pdi
chassis_csv = df_csv_atacado['chassi'].dropna().unique()
chassis_atacado = df_atacado['chassi'].dropna().unique()
# Encontrar os chassis que estão em ambos
chassis_comuns = set(chassis_csv) & set(chassis_atacado)

print(f"Total de chassis no CSV: {len(chassis_csv)}")
print(f"Total de chassis no df_atacado: {len(chassis_atacado)}")
print(f"Chassis que estão em ambos: {len(chassis_comuns)}")
print(f"\nPercentual de chassis do CSV presentes no df_atacado: {len(chassis_comuns) / len(chassis_csv) * 100:.2f}%")

Total de chassis no CSV: 549
Total de chassis no df_atacado: 4201
Chassis que estão em ambos: 548

Percentual de chassis do CSV presentes no df_atacado: 99.82%


In [26]:
# Encontrar chassi em comum
chassis_comuns = set(df_csv_atacado['chassi'].dropna()) & set(df_atacado['chassi'].dropna())
df_csv_comuns = df_csv_atacado[df_csv_atacado['chassi'].isin(chassis_comuns)].copy()

# Ajuste de tipos conforme necessário
df_csv_comuns['pps'] = df_csv_comuns['pps'].str.replace('.', '').str.replace(',', '.').astype(float)
df_csv_comuns['percentil_atacado'] = df_csv_comuns['percentil_atacado'].str.replace(',', '.').astype(float)
df_csv_comuns['dta_recebimento_atacado'] = pd.to_datetime(df_csv_comuns['dta_recebimento_atacado'], format='%d/%m/%Y')

# Defina as colunas que existem tanto no CSV quanto no banco
colunas_banco = set(df_atacado.columns)
colunas_csv = set(df_csv_comuns.columns)
colunas_comuns = list(colunas_banco & colunas_csv)
colunas_comuns.remove('chassi')  # 'chassi' será usado no WHERE

# Atualizar cada registro usando psycopg2
cursor = engine.cursor()
try:
    for _, row in df_csv_comuns.iterrows():
        set_clause = ', '.join([f"{col} = %s" for col in colunas_comuns])
        update_query = f"UPDATE byd.db_compras_byd SET {set_clause} WHERE chassi = %s"
        values = [row[col] for col in colunas_comuns] + [row['chassi']]
        cursor.execute(update_query, values)
    
    engine.commit()
    print(f"Atualização concluída! {len(df_csv_comuns)} registros atualizados.")
except Exception as e:
    engine.rollback()
    print(f"Erro durante atualização: {e}")
    raise
finally:
    cursor.close()

Atualização concluída! 548 registros atualizados.


##### PERFORMANCE

In [29]:
query_performance = """
    SELECT * FROM byd.db_compras_byd """
df_performance = pd.read_sql_query(query_performance, engine)

C:\Users\giovanni.5683\AppData\Local\Temp\ipykernel_22500\272614940.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_performance = pd.read_sql_query(query_performance, engine)


In [30]:
# Carregar o arquivo CSV
df_csv_performance = pd.read_csv('../../../../Datasets/performance.csv')

# Verificar quais chassis do CSV estão no df_pdi
chassis_csv = df_csv_performance['chassi'].dropna().unique()
chassis_performance = df_performance['chassi'].dropna().unique()
# Encontrar os chassis que estão em ambos
chassis_comuns = set(chassis_csv) & set(chassis_performance)

print(f"Total de chassis no CSV: {len(chassis_csv)}")
print(f"Total de chassis no df_performance: {len(chassis_performance)}")
print(f"Chassis que estão em ambos: {len(chassis_comuns)}")
print(f"\nPercentual de chassis do CSV presentes no df_performance: {len(chassis_comuns) / len(chassis_csv) * 100:.2f}%")

Total de chassis no CSV: 1401
Total de chassis no df_performance: 4201
Chassis que estão em ambos: 1398

Percentual de chassis do CSV presentes no df_performance: 99.79%


In [33]:
# Encontrar chassi em comum
chassis_comuns = set(df_csv_performance['chassi'].dropna()) & set(df_performance['chassi'].dropna())
df_csv_comuns = df_csv_performance[df_csv_performance['chassi'].isin(chassis_comuns)].copy()

# Ajuste de tipos conforme necessário
# Converter pps (se existir e não for NaN)
if 'pps' in df_csv_comuns.columns:
    df_csv_comuns['pps'] = df_csv_comuns['pps'].str.replace('R$', '').str.strip().str.replace('.', '').str.replace(',', '.').astype(float)

# Converter percentis (removendo vírgula e convertendo para float)
percentil_cols = ['percentil_tiktok', 'percentil_nps', 'percentil_visits', 
                  'percentil_snapon', 'percentil_mystery', 'percentil_chatbot', 'percentil_download']

for col in percentil_cols:
    if col in df_csv_comuns.columns:
        df_csv_comuns[col] = df_csv_comuns[col].str.replace(',', '.').astype(float)

# Adicionar data de recebimento performance
df_csv_comuns['dta_recebimento_performance'] = pd.Timestamp.now().date()

# Defina as colunas que existem tanto no CSV quanto no banco
colunas_banco = set(df_performance.columns)
colunas_csv = set(df_csv_comuns.columns)
colunas_comuns = list(colunas_banco & colunas_csv)
colunas_comuns.remove('chassi')  # 'chassi' será usado no WHERE

# Atualizar cada registro usando psycopg2
cursor = engine.cursor()
try:
    for _, row in df_csv_comuns.iterrows():
        set_clause = ', '.join([f"{col} = %s" for col in colunas_comuns])
        update_query = f"UPDATE byd.db_compras_byd SET {set_clause} WHERE chassi = %s"
        values = [row[col] for col in colunas_comuns] + [row['chassi']]
        cursor.execute(update_query, values)
    
    engine.commit()
    print(f"Atualização concluída! {len(df_csv_comuns)} registros atualizados.")
except Exception as e:
    engine.rollback()
    print(f"Erro durante atualização: {e}")
    raise
finally:
    cursor.close()

Atualização concluída! 1448 registros atualizados.


In [32]:
conn.rollback()

##### OUTUBRO

In [34]:
query_outubro = """
    SELECT * FROM byd.db_compras_byd """
df_outubro = pd.read_sql_query(query_outubro, engine)

C:\Users\giovanni.5683\AppData\Local\Temp\ipykernel_22500\2505401186.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_outubro = pd.read_sql_query(query_outubro, engine)


In [36]:
# Carregar o arquivo CSV
df_csv_outubro = pd.read_csv('../../../../Datasets/outubro.csv')

# Verificar quais chassis do CSV estão no df_pdi
chassis_csv = df_csv_outubro['chassi'].dropna().unique()
chassis_performance = df_outubro['chassi'].dropna().unique()
# Encontrar os chassis que estão em ambos
chassis_comuns = set(chassis_csv) & set(chassis_performance)

print(f"Total de chassis no CSV: {len(chassis_csv)}")
print(f"Total de chassis no df_outubro: {len(chassis_performance)}")
print(f"Chassis que estão em ambos: {len(chassis_comuns)}")
print(f"\nPercentual de chassis do CSV presentes no df_outubro: {len(chassis_comuns) / len(chassis_csv) * 100:.2f}%")

Total de chassis no CSV: 733
Total de chassis no df_outubro: 4201
Chassis que estão em ambos: 649

Percentual de chassis do CSV presentes no df_outubro: 88.54%


In [37]:
# Encontrar chassi em comum
chassis_comuns = set(df_csv_outubro['chassi'].dropna()) & set(df_outubro['chassi'].dropna())
df_csv_comuns = df_csv_outubro[df_csv_outubro['chassi'].isin(chassis_comuns)].copy()

# Ajuste de tipos conforme necessário
# Converter pps (removendo pontos e vírgulas)
if 'pps' in df_csv_comuns.columns:
    df_csv_comuns['pps'] = df_csv_comuns['pps'].str.replace('.', '').str.replace(',', '.').astype(float)

# Converter val_pdi para float
if 'val_pdi' in df_csv_comuns.columns:
    df_csv_comuns['val_pdi'] = df_csv_comuns['val_pdi'].astype(float)

# Converter percentil_atacado
if 'percentil_atacado' in df_csv_comuns.columns:
    df_csv_comuns['percentil_atacado'] = df_csv_comuns['percentil_atacado'].str.replace(',', '.').astype(float)

# Adicionar data de recebimento de outubro
df_csv_comuns['dta_recebimento_outubro'] = pd.Timestamp.now().date()

# Defina as colunas que existem tanto no CSV quanto no banco
colunas_banco = set(df_outubro.columns)
colunas_csv = set(df_csv_comuns.columns)
colunas_comuns = list(colunas_banco & colunas_csv)

# Remover 'chassi' pois será usado no WHERE
if 'chassi' in colunas_comuns:
    colunas_comuns.remove('chassi')

# Atualizar cada registro usando psycopg2
cursor = engine.cursor()
try:
    for _, row in df_csv_comuns.iterrows():
        set_clause = ', '.join([f"{col} = %s" for col in colunas_comuns])
        update_query = f"UPDATE byd.db_compras_byd SET {set_clause} WHERE chassi = %s"
        values = [row[col] for col in colunas_comuns] + [row['chassi']]
        cursor.execute(update_query, values)
    
    engine.commit()
    print(f"Atualização concluída! {len(df_csv_comuns)} registros atualizados.")
except Exception as e:
    engine.rollback()
    print(f"Erro durante atualização: {e}")
    raise
finally:
    cursor.close()

Atualização concluída! 649 registros atualizados.
